In [ ]:
from transformers import pipeline

classifier = pipeline("token-classification", model="joheras/clinico-bsc-bio-ehr-es-finetuned"); #clinico-bsc-bio-ehr-es-longformer

In [ ]:
import json

with open('clinais.test&background.blind.json') as f:
    data=json.load(f)

In [ ]:
def predice(example):
    res = classifier(example['note_text'])
    track = {}
    for x in res:
        if(x['entity'][0:2]=='B-'):
            track[x['start']]=x['entity'][2:]
    
    
    boundary_annotation = example['boundary_annotation']['gold']
    boundary_annotation_res = []
    for x in boundary_annotation:
        temp = None
        if(x['start_offset'] in track.keys()):
            temp = track[x['start_offset']]
        
        boundary_annotation_res.append({'span':x['span'],'start_offset':x['start_offset'],'end_offset':x['end_offset'],'boundary':temp})
    return boundary_annotation_res

In [ ]:
def predice_v2(example):
    res = classifier(example['note_text'])
    track = {}
    previous = None
    for i,x in enumerate(res):
        if(not('.' in x['word']) and (not('\n' in x['word']))):
            if((x['entity'][0:2]=='B-') and (x['entity'][2:]!=previous)):
                track[x['start']]=x['entity'][2:]
                previous=x['entity'][2:]
            elif(i==0):
                track[x['start']]=x['entity'][2:]
                previous=x['entity'][2:]
            elif(i+1<len(res)):
                if((res[i+1]['entity'][2:]==x['entity'][2:]) and (x['entity'][2:]!=previous)):
                    track[x['start']]=x['entity'][2:]
                    previous=x['entity'][2:]

    
    boundary_annotation = example['boundary_annotation']['gold']
    boundary_annotation_res = []
    for x in boundary_annotation:
        temp = None
        if(x['start_offset'] in track.keys()):
            temp = track[x['start_offset']]
        
        boundary_annotation_res.append({'span':x['span'],'start_offset':x['start_offset'],'end_offset':x['end_offset'],'boundary':temp})
    return boundary_annotation_res

In [5]:
from tqdm import tqdm 
for key in tqdm(data['annotated_entries']):
    data['annotated_entries'][key]['boundary_annotation']['prediction'] = predice_v2(data['annotated_entries'][key])

with open('clinais.test&background.blind.result.json', 'w') as outfile:
    json.dump(data, outfile)

# !python starting_kit/evaluation/evaluate.py -p clinais.dev.result.json -r starting_kit/samples/clinais.dev.json -o result.json --add_scores_in_prediction_file

# with open('result.json') as f:
#     res = json.load(f)
# print(res['Weighted B2'])

100%|██████████| 2843/2843 [16:38<00:00,  2.85it/s]
